In [76]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

from bs4 import BeautifulSoup
import requests
import csv

import datetime as dt
import time

from tqdm.notebook import tqdm, trange
from multiprocessing import Pool

# Weather Data Scraper

## Code

In [3]:
# all weather stations from noaa dataset
weather_stations = pd.read_csv('isd-history.csv')
# countries of interest
country_list = ['CH', 'IN', 'ID', 'BG', 'VM', 'TH', 'BM', 'RP', 'BR', 'JA']
# weather stations filtered by relevant countries
stations_filtered = weather_stations[weather_stations['CTRY'].isin(country_list)].reset_index(drop=True)

In [4]:
url = 'https://www.ncei.noaa.gov/data/daily-summaries/access/'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [8]:
a_tags = soup.find_all('a')

In [35]:
# scraping every csv link from the url
FIRST_CSV_INDEX = 5
csv_links = list(map(lambda x: x.get('href'), a_tags))
csv_links = csv_links[FIRST_CSV_INDEX:]

#dubug
og_len = len(csv_links)

In [36]:
# filtering for target countries by searching for all links that start with 
# target country codes 
COUNTRY_CODE_CUTOFF = 2
filtered_csv_links = list(filter(
    lambda x: x[0:COUNTRY_CODE_CUTOFF] in country_list, csv_links))

#debug
filtered_len = len(filtered_csv_links)
print(f'og_len: {og_len},\n' \
    f'filtered_len: {filtered_len},\n' \
    f'The filtered list is {round(100*(filtered_len/og_len), 2)}% of the original')

og_len: 119468,
filtered_len: 10433,
The filtered list is 8.73% of the original


In [95]:
len(list(filter(lambda x: x[0:2] == 'BR', filtered_csv_links)))

5989

In [96]:
links = filtered_csv_links[6011:]

In [97]:
with requests.Session() as s:    
    for link in tqdm(links): 
        r = s.get(url+link)
        with open('Weather_CSVs/'+link, 'wb') as f:
            f.write(r.content)

  0%|          | 0/4422 [00:00<?, ?it/s]

In [74]:
1.63*11000/60/60

4.980555555555555

In [75]:
pd.read_csv('Weather_CSVs/'+links[1])

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,PRCP,PRCP_ATTRIBUTES,TMAX,TMAX_ATTRIBUTES,TMIN,TMIN_ATTRIBUTES,TAVG,TAVG_ATTRIBUTES
0,BGM00041883,1982-01-01,24.85,89.367,20.0,"BOGRA, BG",NaN,NaN,NaN,NaN,NaN,NaN,172,"H,,S"
1,BGM00041883,1982-01-02,24.85,89.367,20.0,"BOGRA, BG",NaN,NaN,NaN,NaN,NaN,NaN,172,"H,,S"
2,BGM00041883,1982-01-04,24.85,89.367,20.0,"BOGRA, BG",NaN,NaN,NaN,NaN,NaN,NaN,196,"H,,S"
3,BGM00041883,1982-01-05,24.85,89.367,20.0,"BOGRA, BG",NaN,NaN,NaN,NaN,120.0,",,S",172,"H,,S"
4,BGM00041883,1982-01-06,24.85,89.367,20.0,"BOGRA, BG",NaN,NaN,NaN,NaN,NaN,NaN,190,"H,,S"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9609,BGM00041883,2021-02-25,24.85,89.367,20.0,"BOGRA, BG",0.0,",,S",315.0,",,S",187.0,",,S",253,"H,,S"
9610,BGM00041883,2021-02-28,24.85,89.367,20.0,"BOGRA, BG",0.0,",,S",337.0,",,S",203.0,",,S",263,"H,,S"
9611,BGM00041883,2021-03-01,24.85,89.367,20.0,"BOGRA, BG",0.0,",,S",338.0,",,S",196.0,",,S",262,"H,,S"
9612,BGM00041883,2021-11-30,24.85,89.367,20.0,"BOGRA, BG",0.0,",,S",301.0,",,S",NaN,NaN,209,"H,,S"


# Database

In [5]:
try:
    import pyodbc
except:
    %pip install pyodbc
    import pyodbc

In [18]:
server = 'rice-machine.database.windows.net'
database = 'Rice_Price_Prediction'
username = 'Teo'
password = 'D1n0saursrul31?'
driver= '{ODBC Driver 13 for SQL Server}'

cnxn = pyodbc.connect(
    'DRIVER=' + driver \
    + ';PORT=1433;SERVER=' + server \
    + ';PORT=1443;DATABASE=' + database \
    + ';UID=' + username \
    + ';PWD=' + password)
cursor = cnxn.cursor()



Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 13 for SQL Server' : file not found (0) (SQLDriverConnect)")

In [10]:
drivers = [item for item in pyodbc.drivers()]
drivers

[]